In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("./movie_metadata.csv")

In [4]:
data.head(10)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,973.0,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000


In [6]:
# remove unnecessary columns
data2 = data[["num_critic_for_reviews","movie_facebook_likes","duration",
              "director_facebook_likes","actor_3_facebook_likes",
              "actor_2_facebook_likes","actor_1_facebook_likes","gross","num_voted_users",
              "cast_total_facebook_likes","facenumber_in_poster","num_user_for_reviews",
              "budget","imdb_score"]]

In [10]:
data2.dropna(axis=0, how='any')

,num_critic_for_reviews,movie_facebook_likes,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,movie_imdb_link,num_user_for_reviews,budget,imdb_score
0,723.0,33000,178.0,0.0,855.0,936.0,1000.0,760505847.0,886204,4834,0.0,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,237000000.0,7.9
1,302.0,0,169.0,563.0,1000.0,5000.0,40000.0,309404152.0,471220,48350,0.0,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,300000000.0,7.1
2,602.0,85000,148.0,0.0,161.0,393.0,11000.0,200074175.0,275868,11700,1.0,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,245000000.0,6.8
3,813.0,164000,164.0,22000.0,23000.0,23000.0,27000.0,448130642.0,1144337,106759,0.0,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,250000000.0,8.5
5,462.0,24000,132.0,475.0,530.0,632.0,640.0,73058679.0,212204,1873,1.0,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.0,263700000.0,6.6
6,392.0,0,156.0,0.0,4000.0,11000.0,24000.0,336530303.0,383056,46055,0.0,http://www.imdb.com/title/tt0413300/?ref_=fn_t...,1902.0,258000000.0,6.2
7,324.0,29000,100.0,15.0,284.0,553.0,799.0,200807262.0,294810,2036,1.0,http://www.imdb.com/title/tt0398286/?ref_=fn_t...,387.0,260000000.0,7.8
8,635.0,118000,141.0,0.0,19000.0,21000.0,26000.0,458991599.0,462669,92000,4.0,http://www.imdb.com/title/tt2395427/?ref_=fn_t...,1117.0,250000000.0,7.5
9,375.0,10000,153.0,282.0,10000.0,11000.0,25000.0,301956980.0,321795,58753,3.0,http://www.imdb.com/title/tt0417741/?ref_=fn_t...,973.0,250000000.0,7.5
10,673.0,197000,183.0,0.0,2000.0,4000.0,15000.0,330249062.0,371639,24450,0.0,http://www.imdb.com/title/tt2975590/?ref_=fn_t...,3018.0,250000000.0,6.9


In [11]:
data2.corr()

,num_critic_for_reviews,movie_facebook_likes,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,imdb_score
num_critic_for_reviews,1.000000,0.683176,0.258486,0.180674,0.271646,0.282306,0.190016,0.480601,0.624943,0.263203,-0.033897,0.609387,0.119994,0.305303
movie_facebook_likes,0.683176,1.000000,0.196605,0.162048,0.278844,0.243487,0.135348,0.378082,0.537924,0.209786,0.008918,0.400594,0.062039,0.247049
duration,0.258486,0.196605,1.000000,0.173296,0.123558,0.131673,0.088449,0.250298,0.314765,0.123074,0.013469,0.328403,0.074276,0.261662
director_facebook_likes,0.180674,0.162048,0.173296,1.000000,0.120199,0.119601,0.090723,0.144945,0.297057,0.119549,-0.041268,0.221890,0.021090,0.170802
actor_3_facebook_likes,0.271646,0.278844,0.123558,0.120199,1.000000,0.559662,0.249927,0.308026,0.287239,0.473920,0.099368,0.230189,0.047451,0.052633
actor_2_facebook_likes,0.282306,0.243487,0.131673,0.119601,0.559662,1.000000,0.390487,0.262768,0.270790,0.628404,0.071228,0.219496,0.044236,0.083808
actor_1_facebook_likes,0.190016,0.135348,0.088449,0.090723,0.249927,0.390487,1.000000,0.154468,0.192804,0.951661,0.072257,0.145461,0.022639,0.076099
gross,0.480601,0.378082,0.250298,0.144945,0.308026,0.262768,0.154468,1.000000,0.637271,0.247400,-0.027755,0.559958,0.102179,0.198021
num_voted_users,0.624943,0.537924,0.314765,0.297057,0.287239,0.270790,0.192804,0.637271,1.000000,0.265911,-0.026998,0.798406,0.079621,0.410965
cast_total_facebook_likes,0.263203,0.209786,0.123074,0.119549,0.473920,0.628404,0.951661,0.247400,0.265911,1.000000,0.091475,0.206923,0.036557,0.085787
